In [90]:
import numpy as np
import matplotlib.pyplot as plt
import logging
from QLearningEGreedy import QLearningEGreedy as egreedy
import pandas as pd
import seaborn as sns
from pathlib import Path

In [91]:
home = str(Path.home())

f_csma_list = [
    home + "/UFMG/SOMAC-ML/data/_BKP/29092018/csma/round2/backlog_file.npy", # RUN
    home + "/UFMG/SOMAC-ML/data/10102018/round1/csma/backlog_file.npy",      # RUN3
    home + "/UFMG/SOMAC-ML/data/06102018/run4/5/csma/backlog_file.npy"       # RUN4
]

f_tdma_list = [
    home + "/UFMG/SOMAC-ML/data/_BKP/29092018/tdma/round2/backlog_file.npy", # RUN
    home + "/UFMG/SOMAC-ML/data/10102018/round1/tdma/backlog_file.npy",      # RUN3
    home + "/UFMG/SOMAC-ML/data/06102018/run4/5/tdma/backlog_file.npy"       # RUN4
]

In [92]:
def calc_reward(curr, prev):
    # Positive reward
    if curr > prev and curr <= 1.2 * prev:
        reward = 0.
        
    elif curr > 1.2 * prev and curr <= 1.4 * prev:
        reward = 1.
        
    elif curr > 1.4 * prev and curr <= 1.6 * prev:
        reward = 2.
        
    elif curr > 1.6 * prev and curr <= 1.8 * prev:
        reward = 3.
        
    elif curr > 1.8 * prev and curr <= 2.0 * prev:
        reward = 4.
        
    elif curr > 2.0 * prev:
        reward = 5.

    # Negative reward
    elif prev > curr and prev <= 1.2 * curr:
        reward = 0.

    elif prev > 1.2 * curr and prev <= 1.4 * curr:
        reward = -1.
        
    elif prev > 1.4 * curr and prev <= 1.6 * curr:
        reward = -2.
        
    elif prev > 1.6 * curr and prev <= 1.8 * curr:
        reward = -3.
        
    elif prev > 1.8 * curr and prev <= 2.0 * curr:
        reward = -4.
        
    elif prev > 2.0 * curr:
        reward = -5.

    else:
        reward = 0.

    return reward

def test(t_csma = {}, t_tdma = {}, init_prot = 0, learn_rate = 0.3, discount = 0.8, dic = {}, epsilon = 0.1):
    logging.basicConfig(filename="/tmp/out.log", level = logging.INFO)
    
    dic["epsilon"].append(epsilon)
    dic["learn_rate"].append(learn_rate)
    dic["discount"].append(discount)
    
    prot   = init_prot
    somac  = egreedy(prot, learn_rate = learn_rate, discount = discount, epsilon = epsilon)
    metric = {}
    t      = 0
    dt     = -1

    decision = prot
    backlog  = {}
    
    while t < n:
        metric[t] = t_csma[t] if prot == 0 else t_tdma[t]
        
        #print("Metrics = {}".format(metric[t]))
        #print("t = {}, dt = {}".format(t, dt))

        backlog[t] = prot

        if dt > 1:
            if dt == 2:
                reward = calc_reward(metric[t], metric[t-2])
            elif dt == 3:
                reward = calc_reward(metric[t], metric[t-3])
            else:
                reward = calc_reward(metric[t], metric[t-1]) * 0.
                #reward = reward if reward == 0 else reward ** -dt

            somac.update_qtable(reward, dt)

            if dt == 2 and reward > 0:
                decision = somac.decision(prot, keep = True)
            elif dt == 2 and reward < 0.:
                decision = somac.decision(prot, force_switch = True)
            else:
                decision = somac.decision(prot, keep = False)

            if decision != prot:
                dt = 0
        else:
            _ = 0
            #print("MAC prot was switched recently. No switch now")


        t = t + 1
        dt = dt + 1

        #print("Protocol: {}".format(
        #    "CSMA" if prot == 0 else "TDMA"
        #))

        prot = decision

        #print("Decision: {}".format(
        #    "CSMA" if prot == 0 else "TDMA"
        #))

        #print("\n=============================\n")
        
    return backlog, metric
        
        
def calc_stats(backlog, metric, dic = {}):
    n_csma  = 0
    n_tdma  = 0
    n_somac = 0
    n_changes = 0
    
    per_threshold = 0.9
    dic["threshold"].append(per_threshold)
    
    for t in range(n):
        if t_csma[t] >= per_threshold * t_tdma[t]:
            n_csma = n_csma + 1

            if backlog[t] == 0:
                n_somac = n_somac + 1

        if t_tdma[t] >= per_threshold * t_csma[t]:
            n_tdma = n_tdma + 1

            if backlog[t] == 1:
                n_somac = n_somac + 1

    for t in range(1, n, 1):
        if backlog[t-1] != backlog[t]:
            n_changes = n_changes + 1
    
    #########################################################################

    norm = np.max([
        round(np.mean(t_csma), 2),
        round(np.mean(t_tdma), 2),
        round(np.mean([metric[t] for t in range(n)]), 2)
    ])
    
    dic["csma_count"].append(round(n_csma * 1. / n, 2))
    dic["tdma_count"].append(round(n_tdma * 1. / n, 2))
    dic["hit_rate"].append(round(n_somac * 1. / n, 2))
    dic["n_switches"].append(n_changes)
    dic["n_tot"].append(n)
    dic["csma_performance"].append(round(np.mean(t_csma), 2))
    dic["tdma_performance"].append(round(np.mean(t_tdma), 2))
    dic["somac_performance"].append(round(np.mean([metric[t] for t in range(n)]), 2))
    dic["csma_per_performance"].append(round(np.mean(t_csma) / norm, 2))
    dic["tdma_per_performance"].append(round(np.mean(t_tdma) / norm, 2))
    dic["somac_per_performance"].append(round(np.mean([metric[t] for t in range(n)]) / norm, 2))

In [93]:
fname_list = [
    "./_tmp/egreedy/E/run.npy",
    "./_tmp/egreedy/E/run3.npy",
    "./_tmp/egreedy/E/run4.npy"
]

learn_rate = 0.9
discount = 0.8

for fname, f_csma, f_tdma in (zip(fname_list, f_csma_list, f_tdma_list)):
    
    d_csma = np.load(f_csma, encoding = "latin1").item()
    d_tdma = np.load(f_tdma, encoding = "latin1").item()

    n = np.min([len(d_csma), len(d_tdma)]) - 10

    t_csma = np.array([d_csma[t]["metrics"][0, 1] for t in range(n)])
    t_tdma = np.array([d_tdma[t]["metrics"][0, 1] for t in range(n)])
    
    results = {
        "epsilon":               [],
        "learn_rate":            [],
        "discount":              [],
        "threshold":             [],
        "csma_count":            [],
        "tdma_count":            [],
        "hit_rate":              [],
        "n_switches":            [],
        "n_tot":                 [],
        "csma_performance":      [],
        "tdma_performance":      [],
        "somac_performance":     [],
        "csma_per_performance":  [],
        "tdma_per_performance":  [],
        "somac_per_performance": []
    }

    for step in range(10):
        
        prot = 0 if step % 2 == 0 else 1

        if step % 5 == 0:
            print("file: {},\tstep: {}".format(fname, step))

        epsilon_list = [i/100. for i in range(0, 110, 5)]

        for epsilon in epsilon_list:
            backlog, metric = test(
                t_csma = t_csma, t_tdma = t_tdma, init_prot = prot, \
                learn_rate = learn_rate, discount = discount, dic = results, epsilon = epsilon
            )
            calc_stats(backlog, metric, dic = results)
            
    np.save(fname, results)

file: ./_tmp/egreedy/E/run.npy,	step: 0
file: ./_tmp/egreedy/E/run.npy,	step: 5
file: ./_tmp/egreedy/E/run3.npy,	step: 0
file: ./_tmp/egreedy/E/run3.npy,	step: 5
file: ./_tmp/egreedy/E/run4.npy,	step: 0
file: ./_tmp/egreedy/E/run4.npy,	step: 5


In [94]:
df = pd.DataFrame.from_dict(results)

In [95]:
df

,epsilon,learn_rate,discount,threshold,csma_count,tdma_count,hit_rate,n_switches,n_tot,csma_performance,tdma_performance,somac_performance,csma_per_performance,tdma_per_performance,somac_per_performance
0,0.00,0.9,0.8,0.9,0.57,0.51,0.53,1,88,9.59,10.31,10.35,0.93,1.00,1.00
1,0.05,0.9,0.8,0.9,0.57,0.51,0.65,12,88,9.59,10.31,11.37,0.84,0.91,1.00
2,0.10,0.9,0.8,0.9,0.57,0.51,0.65,12,88,9.59,10.31,11.45,0.84,0.90,1.00
3,0.15,0.9,0.8,0.9,0.57,0.51,0.64,14,88,9.59,10.31,11.31,0.85,0.91,1.00
4,0.20,0.9,0.8,0.9,0.57,0.51,0.62,14,88,9.59,10.31,11.27,0.85,0.91,1.00
5,0.25,0.9,0.8,0.9,0.57,0.51,0.67,14,88,9.59,10.31,11.42,0.84,0.90,1.00
6,0.30,0.9,0.8,0.9,0.57,0.51,0.65,16,88,9.59,10.31,11.36,0.84,0.91,1.00
7,0.35,0.9,0.8,0.9,0.57,0.51,0.66,19,88,9.59,10.31,11.02,0.87,0.94,1.00
8,0.40,0.9,0.8,0.9,0.57,0.51,0.66,21,88,9.59,10.31,11.19,0.86,0.92,1.00
9,0.45,0.9,0.8,0.9,0.57,0.51,0.67,21,88,9.59,10.31,11.20,0.86,0.92,1.00
